In [1]:
!pip install konlpy
!pip install gspread
import gspread
from google.colab import drive
drive.mount('/content/drive')

train = '/content/drive/My Drive/mbti/mbti_train.xlsx'  #훈련 데이터가 담긴 엑셀 파일의 구글 드라이브 경로
train = '/content/drive/My Drive/mbti/mbti_test.xlsx'  #테스트 데이터가 담긴 엑셀 파일의 구글 드라이브 경로

!pip install pandas
!pip install matplotlib
!pip install tqdm
!pip install tensorflow
!pip install numpy
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data = pd.read_excel(train)
test_data = pd.read_excel(test)

train_data['내용'] = train_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(train_data))

test_data['내용'] = test_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다',
             '인프피','인픕','인픞','인프제','인티피','인팁','인팊','인티제','잇프피','잇픕','잇픞','잇프제',
             '잇티피','잇팁','잇팊','잇티제','엔프피','엔픕','엔픞','엔프제','엔티피','엔팁','엔팊','엔티제',
             '엣프피','엣픕','엣픞','엣프제','엣티피','엣팁','엣팊','엣티제']

okt = Okt()

X_train = []
for sentence in tqdm(train_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

X_test = []
for sentence in tqdm(test_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)
print(tokenizer.word_counts.items())

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1
tokenizer = Tokenizer(vocab_size) # 빈도수 2 이하인 단어는 제거
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

X_test = np.delete(X_test, drop_test, axis=0)
y_test = np.delete(y_test, drop_test, axis=0)

def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 100
below_threshold_len(max_len, X_train)

# 전체 데이터의 길이는 100으로 맞춘다.
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))


     |████████████████████████████████| 19.4 MB 23.1 MB/s 
     |████████████████████████████████| 86 kB 3.8 MB/s 
     |████████████████████████████████| 448 kB 43.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Mounted at /content/drive
전처리 후 테스트용 샘플의 개수 : 6878
전처리 후 테스트용 샘플의 개수 : 2275


100%|██████████| 2275/2275 [00:13<00:00, 162.89it/s]


{'.': 1, '있다': 2, '저': 3, '같다': 4, '..': 5, '\n': 6, '?': 7, '보다': 8, '제': 9, '요': 10, '것': 11, ',': 12, '사람': 13, '그렇다': 14, '생각': 15, '안': 16, '말': 17, '없다': 18, '되다': 19, '아니다': 20, '...': 21, '!': 22, '좋다': 23, '적': 24, '않다': 25, '그': 26, '거': 27, '때': 28, '자다': 29, '이다': 30, '인': 31, '너무': 32, '해주다': 33, '좋아하다': 34, '다': 35, '로': 36, '하고': 37, '연락': 38, '엔': 39, '내': 40, '싶다': 41, '더': 42, '들다': 43, '프제': 44, '잇다': 45, '못': 46, '나': 47, '고': 48, '분': 49, '맞다': 50, '에서': 51, '만나다': 52, 'ㅎㅎ': 53, '그냥': 54, '게': 55, '받다': 56, '만': 57, '어떻다': 58, '\n ': 59, '친구': 60, '가다': 61, 'ㅠㅠ': 62, 'ㅋㅋㅋ': 63, '많이': 64, '많다': 65, '근데': 66, '님': 67, '일': 68, 'ㅋㅋ': 69, '전': 70, '한테': 71, '(': 72, '마음': 73, '오다': 74, '진짜': 75, ')': 76, '수': 77, '모르다': 78, '랑': 79, '얘기': 80, '인데': 81, '알다': 82, '정말': 83, '이렇다': 84, '아': 85, '정도': 86, '!!': 87, '감사하다': 88, '\n\n': 89, '~': 90, '걸': 91, '시간': 92, '해보다': 93, 'ㅋㅋㅋㅋ': 94, '뭐': 95, '에게': 96, '감정': 97, '표현': 98, '유형': 99, '하': 100, '해': 101, '면': 102, 'ㅠ': 1

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


전체 샘플 중 길이가 100 이하인 샘플의 비율: 96.56432748538012
Epoch 1/15
86/86 [==============================] - ETA: 0s - loss: 0.6891 - acc: 0.5422
Epoch 00001: val_acc improved from -inf to 0.52339, saving model to best_model.h5
86/86 [==============================] - 25s 246ms/step - loss: 0.6891 - acc: 0.5422 - val_loss: 0.6891 - val_acc: 0.5234
Epoch 2/15
86/86 [==============================] - ETA: 0s - loss: 0.6153 - acc: 0.6727
Epoch 00002: val_acc improved from 0.52339 to 0.52632, saving model to best_model.h5
86/86 [==============================] - 20s 237ms/step - loss: 0.6153 - acc: 0.6727 - val_loss: 0.7600 - val_acc: 0.5263
Epoch 3/15
86/86 [==============================] - ETA: 0s - loss: 0.5261 - acc: 0.7346
Epoch 00003: val_acc improved from 0.52632 to 0.53143, saving model to best_model.h5
86/86 [==============================] - 20s 234ms/step - loss: 0.5261 - acc: 0.7346 - val_loss: 0.8142 - val_acc: 0.5314
Epoch 4/15
86/86 [==============================] - ETA: 0s - loss: 0